<a href="https://colab.research.google.com/github/vrublevskiyvitaliy/paraphrase_identification/blob/master/ML_simple_paraphrase_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare data:
We want to get parse trees of sentances.
We will use stanford parser for this.


### This is following the recomendations from:


https://colab.research.google.com/github/stanfordnlp/stanza/blob/master/demo/Stanza_CoreNLP_Interface.ipynb#

In [9]:
!pip install stanza==1.0.0
import stanza

# Download the Stanford CoreNLP Java library and unzip it to a ./corenlp folder
!echo "Downloading CoreNLP..."
!wget "http://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip" -O corenlp.zip
!unzip corenlp.zip
!mv ./stanford-corenlp-full-2018-10-05 ./corenlp

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = "./corenlp"

# Import client module
from stanza.server import CoreNLPClient

# Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
client = CoreNLPClient(annotators=['parse'], memory='4G', endpoint='http://localhost:9001')
print(client)

# Start the background server and wait for some time
# Note that in practice this is totally optional, as by default the server will be started when the first annotation is performed
client.start()
import time; time.sleep(10)

--2020-04-28 16:52:33--  http://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip [following]
--2020-04-28 16:52:33--  https://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 393239982 (375M) [application/zip]
Saving to: ‘corenlp.zip’

corenlp.zip         100%[===================>] 375.02M  11.4MB/s    in 28s     

2020-04-28 16:53:02 (13.3 MB/s) - ‘corenlp.zip’ saved [393239982/393239982]

Archive:  corenlp.zip
   creating: stanford-corenlp-full-2018-10-05/
  inflating: stanford-corenlp-full-2018-10-05/jaxb-core-2.3.0.1-sources.jar  
  inflating: stan

In [0]:
from stanza.protobuf.CoreNLP_pb2 import ParseTree

def parse_sentances(sentances):
  data = []
  for s in sentances:
    document = client.annotate(s)
    tree = document.sentence[0].parseTree
    data.append(tree.SerializeToString())
  return data

def get_parse_tree(sentance):
  document = client.annotate(s)
  return document.sentence[0].parseTree
  # todo: use precomputed data


In [0]:
# Supress output of the cell
%%capture
def download_pickle():
    """
      Downloading pickle
    """ 
    files = [
      'griff4692/paraphrase-detection/contents/vocab/word2Idx.pk',
      'griff4692/paraphrase-detection/contents/vocab/idx2Word.pk',
      'griff4692/paraphrase-detection/contents/vocab/idx2Emb_100.pk',
    ]
    for f in files:
       !curl --remote-name \
          -H 'Accept: application/vnd.github.v3.raw' \
          --location https://api.github.com/repos/{f}

download_pickle()

In [60]:
!wget http://nlp.stanford.edu/data/glove.6B.zip 

--2020-04-28 17:45:38--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-04-28 17:45:38--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-04-28 17:45:38--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

In [0]:
import os
os.environ["StanfordParser"] = "/content/drive/My Drive/phd/stanford-parser-3.9.2-models.jar"



In [0]:
# vocab.py
import numpy as np
from nltk.parse import CoreNLPParser
import os
import pickle

UNK = "<UNK>"

class Vocab:
    WORD2IDX = 'word2Idx.pk'
    IDX2WORD = 'idx2Word.pk'
    IDX2EMB = 'idx2Emb_100.pk'
    
    def __init__(self):
        self.word2Idx = {}
        self.idx2Word = []
        self.idx2Emb = []
        self.parser = CoreNLPParser(url='http://localhost:9001')

    def load(self, embed_size):
        self.word2Idx = pickle.load(open(self.WORD2IDX, 'rb'))
        self.idx2Word = pickle.load(open(self.IDX2WORD, 'rb'))
        self.idx2Emb = pickle.load(open("idx2Emb_100.pk", 'rb'))

    def save(self):
        pickle.dump(self.word2Idx, open(self.WORD2IDX, 'wb'))
        pickle.dump(self.idx2Word, open(self.IDX2WORD, 'wb'))
        pickle.dump(self.idx2Emb, open(self.IDX2EMB, 'wb'))


    def is_fraction(self, string):
        split = string.split('/')

        return len(split) == 2 and split[0].isdigit() and split[1].isdigit()

    def tokenizer(self, word_str):
        word_str = word_str.decode('utf-8')
        parsed_sentence = list(self.parser.raw_parse(word_str))[0].leaves()

        tokens = []
        for (i, token) in enumerate(parsed_sentence):
            token = token.lower()
            if token.isdigit() and i < len(parsed_sentence) - 1:
                if self.is_fraction(parsed_sentence[i + 1]):
                    fraction = token + '-' + parsed_sentence[i + 1]
                    tokens.append(token + '-' + parsed_sentence[i + 1])
                else:
                    tokens.append(token)
            else:
                tokens.append(token)

        return tokens

    def get(self, token):
        return self.word2Idx[token] if token in self.word2Idx else self.unk_idx()

    def get_word(self, idx):
        if idx < 0 or idx >= self.size():
            return UNK
        else:
            return self.idx2Word[idx]

    def tokens_to_idxs(self, tokens):
        return [self.get(token) for token in tokens]

    def sentence_to_idxs(self, sentence_str):
        tokens = self.tokenizer(sentence_str)
        return self.tokens_to_idxs(tokens)

    def build(self, sentences, path_to_embed, emb_dim):
        self.emb_dim = emb_dim
        self.max_sent_length = 0
        for sentence in sentences:
            tokens_1 = self.tokenizer(sentence[0])
            tokens_2 = self.tokenizer(sentence[1])

            len1 = len(tokens_1)
            len2 = len(tokens_2)

            self.max_sent_length = np.max([len1, len2, self.max_sent_length])

            for token in tokens_1 + tokens_2:
                if token not in self.word2Idx:
                    idx = len(self.idx2Word)
                    self.idx2Word.append(token)
                    self.word2Idx[token] = idx

        self.build_emb_matrix(path_to_embed, emb_dim)

    def unk_idx(self):
        return self.size()

    def size(self):
        return len(self.idx2Word)

    def build_emb_matrix(self, path_to_embed, emb_dim):
        self.emb_dim = emb_dim
        self.idx2Emb = np.zeros((self.size() + 1, emb_dim), dtype='float')

        hits = 0

        embeddings = open(path_to_embed)
        for embedding in embeddings:
            split = embedding.split()
            word = split[0]

            if word in self.word2Idx:
                vals =  np.array([float(num) for num in split[1:]])
                self.idx2Emb[self.word2Idx[word]] = vals
                hits += 1

        print("%s out of %s word embeddings initialized." % (hits, self.size()))

In [0]:
# from nltk.parse.stanford import StanfordParser
from nltk.parse import CoreNLPParser
import os
from queue import Queue

parser = CoreNLPParser(url='http://localhost:9001')


def glob_compound_leaves(leaves, compound1, compound2 = None):
    if compound2 is None:
        return leaves

    for (i, leaf) in enumerate(leaves):
        if compound1 == leaf and compound2 == leaves[i + 1]:
            # both are equal to compound
            leaves[i] = compound1 + "-" + compound2
            leaves = leaves[:i + 1] + leaves[i + 2:]
            return leaves

    raise Exception("Should be able to find compount leaf words!")


def build_tree(raw_sent, vocab):
    tree = list(parser.raw_parse(raw_sent))[0]

    leaves = [leaf.lower() for leaf in tree.leaves()]

    id_counter = 0

    rootNode = TreeNode(tree.label(), tree.height() - 1, id_counter, -1, None, None)
    sentence = TreeSentence(rootNode)

    q = Queue()
    q.put((tree, 0, id_counter)) # nltk Tree Object, level, current_id

    while not q.empty():
        curr_subtree, curr_level, curr_id = q.get()

        for subtree in curr_subtree:
            id_counter += 1

            subtree_node = TreeNode(subtree.label(), subtree.height() - 1, id_counter, curr_id, None, None)
            node_idx = sentence.add_to_level(curr_level + 1, subtree_node)

            if type(subtree[0]) == unicode:
                word = subtree[0].lower()
                if len(subtree) > 1:
                    word2 = subtree[1].lower()
                    leaves = glob_compound_leaves(leaves, word, word2)
                    word += "-" + word2

                subtree_node.word = word
                subtree_node.token = vocab.get(word)
            else:
                q.put((subtree, curr_level + 1, id_counter))

    sentence.add_leaves(leaves, vocab)
    return sentence


class TreeSentence:
    def __init__ (self, levels, leaves = []):
        self.leaves = leaves
        if isinstance(levels, TreeNode):
            root = levels
            self.levels = [[] for i in range(root.get_height())]
            self.levels[0].append(root)
        else:
            self.levels = levels
            for (r, level) in enumerate(levels):
                for(c, node_dict) in enumerate(level):
                    if(not isinstance(node_dict, TreeNode)):
                        self.levels[r][c] = TreeNode(**node_dict)

    def add_to_level(self, idx, node):
        self.levels[idx].append(node)
        return len(self.levels[idx]) - 1

    def add_leaves(self, leaves, vocab):
        self.leaves = [vocab.get(leaf) for leaf in leaves]

    def get_height(self):
        return self.levels[0][0].get_height()

    def render(self, vocab=None):
        for (i, level) in enumerate(self.levels):
            print("Level #" + str(i) + "\n")
            for (j, node) in enumerate(level):
                node.pretty_print(self.get_children(i, j), vocab)
            print ("\n")

    def get_children(self, level, idx):
        parent_node = self.levels[level][idx]

        if level >= len(self.levels) - 1:
            return []
        else:
            return [
                (i, node) for (i, node) in enumerate(self.levels[level + 1])
                    if node.parent_id == parent_node.my_id
            ]

class TreeNode:
    def __init__(self, label, height, my_id, parent_id, token, word):
        self.label = label
        self.token = token
        self.word = word
        self.my_id = my_id
        self.height = height
        self.parent_id = parent_id

    def get_height(self):
        return self.height

    def is_leaf(self):
        return self.get_height() == 1

    def pretty_print(self, child_ids, vocab=None):
        child_ids = [str(child_id[0]) for child_id in child_ids]
        print ('ID = ( ' + str(self.my_id) + ' )')
        print ('\t--> Label = ' + self.label)
        print ('\t--> Height = ' + str(self.height))
        print ('\t--> Parent ID = ( ' + str(self.parent_id) + ' )')
        print ('\t--> Child IDs = [ ' + ", ".join(child_ids) + ' ]')
        if self.word is not None:
            print ('\t--> Word = ' + self.word + ' (' + str(self.token) + ')')

# if __name__== '__main__':
#     sentence = "the cats from new york catch mice".decode('utf-8')
#     vocab = Vocab()
#     vocab.load(100)
#     tree_sentence = build_tree(sentence, vocab)
#     tree_sentence.render(vocab)

In [58]:
sentence = "the cats from new york catch mice"
vocab = Vocab()
vocab.load(100)
tree_sentence = build_tree(sentence, vocab)
tree_sentence.render(vocab)

UnpicklingError: ignored